In [53]:
import ast
import numpy as np
import pandas as pd

from opportunistic_planning import prediction, processing
from scipy.stats import wilcoxon

In [23]:
data = pd.read_csv('all_task_environments_list_2022-09-12_unique_sequences.csv', header=0)

In [32]:
ts = data[:168]

In [33]:
distances_dict_kit = processing.generate_distances_dict(ts, use_string_for_seq=False,
                                                   dimensions=[[2, 'xy']])

### test alternative value ranges

In [34]:
def calculate_prediction_error(data, distances_dict, error_function, n=10, 
                             dimensions=[[2, 'xy'], [3, 'xyz']], 
                             seqcol='sequence', coords='coordinates', error='error',
                             use_string_for_seq=False):

    results = pd.DataFrame()
    
    for row in range(0, len(data)):
        # get episode information from input row
        coordinates = {key: ast.literal_eval(value) for key, value in
                       (elem.split(': ') for elem in data.at[row, coords].split(';'))}

        start_coordinates = list(ast.literal_eval(data.at[row, 'start_coordinates']))
        ID = str(data.at[row,'ID'])
        
        if use_string_for_seq == True:
            seq = str(data.at[row, seqcol])
            objects = list(data.at[row, seqcol])
        else:
            seq = [elem for elem in data.at[row, seqcol].split(',')]
            objects = [elem for elem in data.at[row, seqcol].split(',')]

        # get list of objects that have relational dependencies, if any (else set to empty list)
        try:
            strong_k = list(data.at[row, 'strong_k'].split(','))
        except AttributeError:
            strong_k = []

        try:
            mid_k = list(data.at[row, 'mid_k'].split(','))
        except AttributeError:
            mid_k = []

        try:
            food_k = list(data.at[row, 'food_k'].split(','))
        except AttributeError:
            food_k = []
            
        try:
            containment = list(data.at[row, 'containment']).split(',')
        except AttributeError:
            containment = []

        # go through parameter ranges
        # set k to current param if object has relational dependencies, else 1.0
        for k2 in np.arange(2.1, 3.0, 0.1):
            k_food = round(k2, 2)
            k1 = {obj: k_food if obj in food_k else 1.0 for obj in objects}

            for k in np.arange(-1.0, 0, 0.1):
                k_strong = round(k, 2)
                k_mid = round(k + 0.1, 2)
                k1 = {obj: k_strong if obj in strong_k else k_mid if obj in mid_k else round(k1[obj], 2) for obj in
                      objects}

                for c in np.arange(2.0, 3.0, 0.1):
                    c = round(c, 1)
                    # set c to current param if object contained, else 1.0
                    c1 = {obj: c if obj in containment else 1.0 for obj in objects}

                    for dim in dimensions:
                        # get median error for parameter combination based on error function
                        median = prediction.get_median_error(error_function, row, ID, objects, 
                                                             coordinates, start_coordinates, 
                                                             c1, k1, dim,
                                                             seq, distances_dict, n)

                        # save parameter combination as column name in results
                        params = 'c: ' + str(c) + '; k: ' + str(k_strong) + ',' + str(k_mid) + ',' + str(
                            k_food) + '; ' + str(dim[1])

                        results.at[row, params] = median

        #results.at[row, 'sequence'] = seq
        results.at[row, 'error'] = data.at[row, error]
        results.at[row, 'ID'] = ID

    return results

In [35]:
results = calculate_prediction_error(ts, distances_dict_kit, error_function='prequential',
                                    n=100, dimensions=[[2, 'xy']], seqcol='sequence',
                                    coords='coordinates', error='error',
                                    use_string_for_seq=False)

/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe =

/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe =

/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe =

/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe =

/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe =

/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe =

/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe =

/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe =

/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results.at[row, params] = median
/tmp/ipykernel_452920/78799914.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe =

/home/pwenzl/personal/projects/phd thesis/cognitive model/model_package/opportunistic_planning/prediction.py:176: RuntimeWarning: divide by zero encountered in scalar power
  possible_items[obj] = distances_dict[dimension[1]][ID][position][obj] ** k[obj] * c[obj]


In [38]:
lowest_mean, lowest_mean_idx, lowest_median, results_median = processing.get_lowest_error(results)

In [41]:
lowest_mean, lowest_median, lowest_mean_idx

(4.083333333333333,
 4.0,
 Index(['c: 2.4; k: -0.8,-0.7,2.1; xy', 'c: 2.8; k: -0.6,-0.5,2.4; xy'], dtype='object'))

In [48]:
processing.save_results(results, 'results/results_alternative_parameter_ranges_n100_2D_2024-02-20.csv')

### Compare to previous (normal range) results

In [49]:
results_old = pd.read_csv('results/results_tablesetting_2D_n100_fitted_to_ts_2023-06-22.csv', header=0)

In [50]:
results_old

,"c: 1.0; k: 0.0,0.1,1.1; xy","c: 1.1; k: 0.0,0.1,1.1; xy","c: 1.2; k: 0.0,0.1,1.1; xy","c: 1.3; k: 0.0,0.1,1.1; xy","c: 1.4; k: 0.0,0.1,1.1; xy","c: 1.5; k: 0.0,0.1,1.1; xy","c: 1.6; k: 0.0,0.1,1.1; xy","c: 1.7; k: 0.0,0.1,1.1; xy","c: 1.8; k: 0.0,0.1,1.1; xy","c: 1.9; k: 0.0,0.1,1.1; xy",...,"c: 1.2; k: 0.8,0.9,1.9; xy","c: 1.3; k: 0.8,0.9,1.9; xy","c: 1.4; k: 0.8,0.9,1.9; xy","c: 1.5; k: 0.8,0.9,1.9; xy","c: 1.6; k: 0.8,0.9,1.9; xy","c: 1.7; k: 0.8,0.9,1.9; xy","c: 1.8; k: 0.8,0.9,1.9; xy","c: 1.9; k: 0.8,0.9,1.9; xy",error,ID
0,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,4.00000,4.000000,4.00000,4.000000,4.00000,4.000000,4.000000,4.000000,0.723,a1
1,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,...,7.00000,7.000000,7.00000,7.000000,7.00000,7.000000,7.000000,7.000000,0.785,a3
2,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,...,4.00000,4.000000,4.00000,4.000000,4.00000,4.000000,4.000000,4.000000,0.806,a5
3,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,7.00000,7.000000,7.00000,7.000000,7.00000,7.000000,7.000000,7.000000,0.862,a11
4,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,...,6.00000,6.000000,6.00000,6.000000,6.00000,6.000000,6.000000,6.000000,0.806,a13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,2.000000,2.000000,3.000000,3.000000,2.000000,2.000000,3.000000,3.000000,2.000000,2.000000,...,2.00000,3.000000,2.50000,2.000000,2.00000,2.000000,2.500000,2.000000,0.723,v7
184,3.500000,4.000000,3.500000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,...,4.00000,4.000000,3.00000,4.000000,3.00000,4.000000,4.000000,3.000000,0.723,v8
185,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,...,2.00000,2.000000,3.00000,3.000000,2.00000,2.000000,3.000000,3.000000,0.723,v9
186,4.180108,4.139785,4.137097,4.137097,4.096774,4.142473,4.147849,4.150538,4.123656,4.155914,...,4.63172,4.615591,4.66129,4.602151,4.61828,4.642473,4.629032,4.634409,NaN,NaN


In [51]:
lowest_mean_old, lowest_mean_idx_old, lowest_median_old, results_median_old = processing.get_lowest_error(results_old)

In [52]:
lowest_mean_old, lowest_median_old, lowest_mean_idx_old

(4.0427819720887666,
 4.0,
 Index(['c: 1.7; k: 0.2,0.3,1.2; xy'], dtype='object'))

In [63]:
lowest_new = results['c: 2.4; k: -0.8,-0.7,2.1; xy'].tolist()
len(lowest_new)

170

In [65]:
lowest_old = results_old['c: 1.7; k: 0.2,0.3,1.2; xy'].tolist()[:-20]
len(lowest_old)

170

In [66]:
stat, p = wilcoxon(lowest_new, lowest_old, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 2138.000, p = 0.56940
